In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive//MyDrive/Colab\ Notebooks/Advanced_programming_for_scientific_computing/
# Master_homework

ls: cannot access '/content/drive//MyDrive/Colab Notebooks/Advanced_programming_for_scientific_computing/': No such file or directory


In [ ]:
# !unrar x /content/drive/MyDrive/Colab\ Notebooks/Master_homework/Advanced_programming_for_scientific_computing/cropped.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/Colab Notebooks/Master_homework/Advanced_programming_for_scientific_computing/cropped.rar


Would you like to replace the existing file cropped/A10_100_cropped.jpg
  3404 bytes, modified on 2024-03-19 13:12
with a new one
  3404 bytes, modified on 2024-03-19 13:12

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit Q

Program aborted


In [ ]:
!pip install timm

NameError: name 'vectors' is not defined

In [ ]:
import os
from imageio import imread
import torch
import numpy as np
import timm
from glob import glob
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import cv2
import pandas as pd
from PIL import Image




class CroppedDataset(Dataset):
    """Face Landmarks dataset."""
    def __init__(self, img_list, img_size = 224, transforms = None):
        self.img_list = img_list
        self.img_size = img_size
        self.transforms = transforms
    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):

        file = self.img_list[idx]
        img_name  = os.path.basename(file).split("_cropped")[0]
        img =  imread(file)
        img = cv2.resize(img, (self.img_size, self.img_size))/255
        img = img.reshape(3, img.shape[0],img.shape[1])
        # img = Image.open(file)
        # img = self.transforms(img)
        return img_name, img

model_names = {
    "timm/vit_large_patch14_dinov2.lvd142m": 518,
    "timm/vit_base_patch16_224.dino": 224,
    "timm/vit_small_patch14_dinov2.lvd142m": 518,
    "timm/eva_large_patch14_196.in22k_ft_in22k_in1k": 196,
    "timm/vit_base_patch14_dinov2.lvd142m": 518,
    # "timm/samvit_base_patch16.sa1b": 1024,
    "timm/vit_base_patch16_224.mae": 224,
    'resnet34': 224
}



img_list = glob("./cropped/*.jpg")

for model_name, img_size in model_names.items():
  names = []
  vectors = []
  pretrained = timm.create_model(model_name, pretrained=True, num_classes=0).to("cuda")
  data_config = timm.data.resolve_model_data_config(pretrained)
  del data_config["crop_pct"]
  del data_config["crop_mode"]
  transforms = timm.data.create_transform(**data_config, is_training=False)

  dataset = CroppedDataset(img_list, img_size = img_size, transforms = transforms)
  dataloader = DataLoader(dataset, batch_size=2,
                          shuffle=False, num_workers=2)

  for batch_names, batch_images in tqdm(dataloader):
    names.extend(batch_names)
    img_vector = pretrained(batch_images.float().to("cuda"))
    vectors.append(img_vector.detach().to("cpu").numpy())

  del pretrained
  vectors = np.concatenate(vectors)
  orange_name = np.array([name.split("_")[0] for name in names])
  orange_image_id = np.array([int(name.split("_")[1]) for name in names])

  name_set  = list(set(orange_name))

  options = ["mean", "get_one", "get_97"] + [f"get_{k}" for k in [5, 10, 20, 30]] + [f"interval_{k}" for k in [5, 10, 20, 30]]
  # options = [f"interval_{k}" for k in [3, 4, 5, 10, 15, 20, 30]]
  for option in options:
    orange_mean_vectors = []
    for name in name_set:
      index  = np.where(orange_name == name)[0]
      selected_id = orange_image_id[index]
      selected_vector = vectors[index]
      sort_order = np.argsort(selected_id)

      sorted_vector = selected_vector[sort_order]
      # print(selected_id[sort_order])
      # print(sorted_vector)

      if option == "mean":
          orange_vector = np.mean(sorted_vector, axis = 0)
          orange_mean_vectors.append(orange_vector)
      elif option == "get_one":
          orange_vector = sorted_vector[0]
          orange_mean_vectors.append(orange_vector)
      elif option == "get_97":
          orange_vector = np.mean(sorted_vector[:97], axis = 0)
          orange_mean_vectors.append(orange_vector)
      elif "interval" in option:
        k = int(option.split("_")[-1])
        index = np.arange(0, 97, k)
        orange_vector = np.mean(sorted_vector[index], axis = 0)
        orange_mean_vectors.append(orange_vector)
      else:
        k = int(option.split("_")[-1])
        orange_vector = np.mean(sorted_vector[:k], axis = 0)
        orange_mean_vectors.append(orange_vector)


    name_feats = [f"{model_name}_{i}" for i in range(len(orange_mean_vectors[0]))]
    vecs = np.vstack(orange_mean_vectors)
    df_resnet = pd.DataFrame(data = vecs, columns = name_feats)
    df_name = pd.DataFrame({"name": name_set})
    df = pd.concat([df_name, df_resnet], axis =1)
    save_name = model_name.split("/")[-1]
    df.to_excel(f"/content/drive//MyDrive/Colab Notebooks/Master_homework/Advanced_programming_for_scientific_computing/timm/{save_name}_{option}.xlsx", index = False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
100%|██████████| 4495/4495 [47:51<00:00,  1.57it/s]


model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

100%|██████████| 4495/4495 [02:07<00:00, 35.32it/s]


model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

100%|██████████| 4495/4495 [05:18<00:00, 14.10it/s]


...

100%|██████████| 4495/4495 [15:13<00:00,  4.92it/s]


model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

100%|██████████| 4495/4495 [02:09<00:00, 34.74it/s]


model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

100%|██████████| 4495/4495 [01:06<00:00, 67.71it/s]


In [2]:
len(selected_feats_backup)

707